###【 그룹화 연산 】

- 필터링과 반복문을 통한 관련 데이터 추출 과정을 진행하는 메서드 제공
- 메서드 : groupby(컬럼명 or [컬럼명, 컬럼명])
- 객체 : DataFrameGroupBy / SeriesGroupBy
- 속성 
    * groups => dict 반환. 그룹키:해당인덱스
    * ngroups => 그룹 개수
    * size() => 각 그룹형 행 개수
    * get_group(그룹명/그룹키) => 해당 그룹의 DataFrames/Series
    

[1] 모듈 로딩 및 데이터 준비 <hr>

In [1]:
## 모듈 로딩
import pandas as pd

In [2]:
## DataFrame 인스턴스 생성
df = pd.DataFrame({
    '지점': ['서울','서울','서울','부산','부산','대구','대구','대구'],
    '상품': ['A','A','B','A','B','A','B','B'],
    '매출': [100,150,200,120,180,90,210,240],
    '수량': [10,15,20,12,18,9,21,24]
})

## 구조 확인
df.info()
display( df )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지점      8 non-null      object
 1   상품      8 non-null      object
 2   매출      8 non-null      int64 
 3   수량      8 non-null      int64 
dtypes: int64(2), object(2)
memory usage: 388.0+ bytes


,지점,상품,매출,수량
0,서울,A,100,10
1,서울,A,150,15
2,서울,B,200,20
3,부산,A,120,12
4,부산,B,180,18
5,대구,A,90,9
6,대구,B,210,21
7,대구,B,240,24


[2] 그룹화 진행 및 기본 속성들 <hr>

In [3]:
## 지점별 그룹화 
g = df.groupby('지점')

print(type(g))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


In [4]:
## 그룹 속성 확인
print(f"-> 그룹수 : { g.ngroups}개")        # 그룹 수
print(f"-> 그룹별 : {g.groups}" )           # 각 그룹의 인덱스
print(f"-> 그룹별 행 수\n{g.size()}" )       # 그룹별 행 수


-> 그룹수 : 3개
-> 그룹별 : {'대구': [5, 6, 7], '부산': [3, 4], '서울': [0, 1, 2]}
-> 그룹별 행 수
지점
대구    3
부산    2
서울    3
dtype: int64


In [5]:
## 그룹별 데이터 추출
for k in g.groups.keys():
    print(f'\n[{k}]=====')
    print(g.get_group(k))       


[대구]=====
   지점 상품   매출  수량
5  대구  A   90   9
6  대구  B  210  21
7  대구  B  240  24

[부산]=====
   지점 상품   매출  수량
3  부산  A  120  12
4  부산  B  180  18

[서울]=====
   지점 상품   매출  수량
0  서울  A  100  10
1  서울  A  150  15
2  서울  B  200  20


[3] 그룹별 집계 메서드 <hr>

In [6]:
## (1) 그룹별 합계
print("합계===>\n", g.sum(numeric_only=True))

## (2) 평균
print("\n평균===>\n", g.mean(numeric_only=True))
print()

## (3) 여러 집계 동시 적용
print(g.agg({'매출':['sum','mean'], '수량':['max','min']}))

합계===>
      매출  수량
지점         
대구  540  54
부산  300  30
서울  450  45

평균===>
        매출    수량
지점             
대구  180.0  18.0
부산  150.0  15.0
서울  150.0  15.0

     매출         수량    
    sum   mean max min
지점                    
대구  540  180.0  24   9
부산  300  150.0  18  12
서울  450  150.0  20  10


In [7]:
## 지점의 상품별 매출 합계와 평균 
for key in g.groups.keys():
    print(f'\n[{key}]=====')
    localDF = g.get_group(key).drop(columns='지점').reset_index(drop=True)
    
    localDF=localDF.groupby('상품')
    display( localDF.agg({'매출':['sum','mean']}) )


[대구]=====


매출       
    sum   mean
상품            
A    90   90.0
B   450  225.0


[부산]=====


매출       
    sum   mean
상품            
A   120  120.0
B   180  180.0


[서울]=====


매출       
    sum   mean
상품            
A   250  125.0
B   200  200.0

In [8]:
## 지점별 상품에 따른 매출 합계와 평균 집계 
g = df.groupby(['지점','상품'])

## 그룹별 데이터 추출
for k in g.groups.keys():
    print(f'\n[{k}]=====')
    print(g.get_group(k))       


[('대구', 'A')]=====
   지점 상품  매출  수량
5  대구  A  90   9

[('대구', 'B')]=====
   지점 상품   매출  수량
6  대구  B  210  21
7  대구  B  240  24

[('부산', 'A')]=====
   지점 상품   매출  수량
3  부산  A  120  12

[('부산', 'B')]=====
   지점 상품   매출  수량
4  부산  B  180  18

[('서울', 'A')]=====
   지점 상품   매출  수량
0  서울  A  100  10
1  서울  A  150  15

[('서울', 'B')]=====
   지점 상품   매출  수량
2  서울  B  200  20


In [9]:
## 지점별 상품에 따른 매출 합계, 평균
g.agg(['sum','mean'])

매출         수량      
       sum   mean sum  mean
지점 상품                      
대구 A    90   90.0   9   9.0
   B   450  225.0  45  22.5
부산 A   120  120.0  12  12.0
   B   180  180.0  18  18.0
서울 A   250  125.0  25  12.5
   B   200  200.0  20  20.0

[4] 그룹별 계산한 결과 ==> 원래 데이터의 행 개수 그대로 반환 <hr>
- 활용 : 그룹 평균 대비 비율, 표준화, 순위 계산, z-score

In [10]:
## DataFrame 인스턴스 생성
df = pd.DataFrame({
    '지점': ['서울','서울','서울','부산','부산','부산','대구','대구','대구'],
    '매출': [100,150,200,120,180,160,90,210,240]
})

## 확인
display(df)

,지점,매출
0,서울,100
1,서울,150
2,서울,200
3,부산,120
4,부산,180
5,부산,160
6,대구,90
7,대구,210
8,대구,240


In [11]:
## ------------------------------------------
## (1) 그룹별 평균 매출 대비 비율
## ------------------------------------------
df['매출비율'] = df.groupby('지점')['매출'].transform(lambda x: x / x.mean())

## 같은 지점 내에서 매출이 평균보다 높은지/낮은지를 한눈에 볼 수 있음.
display(df)

,지점,매출,매출비율
0,서울,100,0.666667
1,서울,150,1.000000
2,서울,200,1.333333
3,부산,120,0.782609
4,부산,180,1.173913
5,부산,160,1.043478
6,대구,90,0.500000
7,대구,210,1.166667
8,대구,240,1.333333


In [12]:
## =================================================================
## (2) 그룹별 표준화 (z-score) 
##     => 데이터의 상대적인 크기를 맞추어 공정 비교할 수 있도록 하는 변환
## =================================================================
## - 같은 지점 내에서 평균보다 높은 값은 +, 낮은 값은 -
## - 각 지점별 데이터의 상대적 위치(편차)를 확인할 수 있음
## - 모든 지점별로 평균은 0, 표준편차는 1로 맞춰짐
df['표준화'] = df.groupby('지점')['매출'].transform(lambda x: (x - x.mean()) / x.std())
display(df)

,지점,매출,매출비율,표준화
0,서울,100,0.666667,-1.000000
1,서울,150,1.000000,0.000000
2,서울,200,1.333333,1.000000
3,부산,120,0.782609,-1.091089
4,부산,180,1.173913,0.872872
5,부산,160,1.043478,0.218218
6,대구,90,0.500000,-1.133893
7,대구,210,1.166667,0.377964
8,대구,240,1.333333,0.755929


In [13]:
## ------------------------------------------
## (3) 그룹별 순위(rank)
## ------------------------------------------
df['순위'] = df.groupby('지점')['매출'].transform(lambda x: x.rank(ascending=False))
display(df)

,지점,매출,매출비율,표준화,순위
0,서울,100,0.666667,-1.000000,3.0
1,서울,150,1.000000,0.000000,2.0
2,서울,200,1.333333,1.000000,1.0
3,부산,120,0.782609,-1.091089,3.0
4,부산,180,1.173913,0.872872,1.0
5,부산,160,1.043478,0.218218,2.0
6,대구,90,0.500000,-1.133893,3.0
7,대구,210,1.166667,0.377964,2.0
8,대구,240,1.333333,0.755929,1.0
